In [1]:
# allowable multiple choice node and edge features 
allowable_features = {
    'possible_atomic_num_list' : list(range(1, 119)) + ['misc'],
    'possible_chirality_list' : [
        'CHI_UNSPECIFIED',
        'CHI_TETRAHEDRAL_CW',
        'CHI_TETRAHEDRAL_CCW',
        'CHI_OTHER',
        'misc'
    ],
    'possible_degree_list' : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'misc'],
    'possible_formal_charge_list' : [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 'misc'],
    'possible_numH_list' : [0, 1, 2, 3, 4, 5, 6, 7, 8, 'misc'],
    'possible_number_radical_e_list': [0, 1, 2, 3, 4, 'misc'],
    'possible_hybridization_list' : [
        'SP', 'SP2', 'SP3', 'SP3D', 'SP3D2', 'misc'
        ],
    'possible_is_aromatic_list': [False, True],
    'possible_is_in_ring_list': [False, True],
    'possible_bond_type_list' : [
        'SINGLE',
        'DOUBLE',
        'TRIPLE',
        'AROMATIC',
        'misc'
    ],
    'possible_bond_stereo_list': [
        'STEREONONE',
        'STEREOZ',
        'STEREOE',
        'STEREOCIS',
        'STEREOTRANS',
        'STEREOANY',
    ], 
    'possible_is_conjugated_list': [False, True],
}

def safe_index(l, e):
    """
    Return index of element e in list l. If e is not present, return the last index
    """
    try:
        return l.index(e)
    except:
        return len(l) - 1
# # miscellaneous case
# i = safe_index(allowable_features['possible_atomic_num_list'], 'asdf')
# assert allowable_features['possible_atomic_num_list'][i] == 'misc'
# # normal case
# i = safe_index(allowable_features['possible_atomic_num_list'], 2)
# assert allowable_features['possible_atomic_num_list'][i] == 2

def atom_to_feature_vector(atom):
    """
    Converts rdkit atom object to feature list of indices
    :param mol: rdkit atom object
    :return: list
    """
    atom_feature = [
            safe_index(allowable_features['possible_atomic_num_list'], atom.GetAtomicNum()),
            safe_index(allowable_features['possible_chirality_list'], str(atom.GetChiralTag())),
            safe_index(allowable_features['possible_degree_list'], atom.GetTotalDegree()),
            safe_index(allowable_features['possible_formal_charge_list'], atom.GetFormalCharge()),
            safe_index(allowable_features['possible_numH_list'], atom.GetTotalNumHs()),
            safe_index(allowable_features['possible_number_radical_e_list'], atom.GetNumRadicalElectrons()),
            safe_index(allowable_features['possible_hybridization_list'], str(atom.GetHybridization())),
            allowable_features['possible_is_aromatic_list'].index(atom.GetIsAromatic()),
            allowable_features['possible_is_in_ring_list'].index(atom.IsInRing()),
            ]
    return atom_feature
# from rdkit import Chem
# mol = Chem.MolFromSmiles('Cl[C@H](/C=C/C)Br')
# atom = mol.GetAtomWithIdx(1)  # chiral carbon
# atom_feature = atom_to_feature_vector(atom)
# assert atom_feature == [5, 2, 4, 5, 1, 0, 2, 0, 0]


def get_atom_feature_dims():
    return list(map(len, [
        allowable_features['possible_atomic_num_list'],
        allowable_features['possible_chirality_list'],
        allowable_features['possible_degree_list'],
        allowable_features['possible_formal_charge_list'],
        allowable_features['possible_numH_list'],
        allowable_features['possible_number_radical_e_list'],
        allowable_features['possible_hybridization_list'],
        allowable_features['possible_is_aromatic_list'],
        allowable_features['possible_is_in_ring_list']
        ]))

def bond_to_feature_vector(bond):
    """
    Converts rdkit bond object to feature list of indices
    :param mol: rdkit bond object
    :return: list
    """
    bond_feature = [
                safe_index(allowable_features['possible_bond_type_list'], str(bond.GetBondType())),
                allowable_features['possible_bond_stereo_list'].index(str(bond.GetStereo())),
                allowable_features['possible_is_conjugated_list'].index(bond.GetIsConjugated()),
            ]
    return bond_feature
# uses same molecule as atom_to_feature_vector test
# bond = mol.GetBondWithIdx(2)  # double bond with stereochem
# bond_feature = bond_to_feature_vector(bond)
# assert bond_feature == [1, 2, 0]

def get_bond_feature_dims():
    return list(map(len, [
        allowable_features['possible_bond_type_list'],
        allowable_features['possible_bond_stereo_list'],
        allowable_features['possible_is_conjugated_list']
        ]))

def atom_feature_vector_to_dict(atom_feature):
    [atomic_num_idx, 
    chirality_idx,
    degree_idx,
    formal_charge_idx,
    num_h_idx,
    number_radical_e_idx,
    hybridization_idx,
    is_aromatic_idx,
    is_in_ring_idx] = atom_feature

    feature_dict = {
        'atomic_num': allowable_features['possible_atomic_num_list'][atomic_num_idx],
        'chirality': allowable_features['possible_chirality_list'][chirality_idx],
        'degree': allowable_features['possible_degree_list'][degree_idx],
        'formal_charge': allowable_features['possible_formal_charge_list'][formal_charge_idx],
        'num_h': allowable_features['possible_numH_list'][num_h_idx],
        'num_rad_e': allowable_features['possible_number_radical_e_list'][number_radical_e_idx],
        'hybridization': allowable_features['possible_hybridization_list'][hybridization_idx],
        'is_aromatic': allowable_features['possible_is_aromatic_list'][is_aromatic_idx],
        'is_in_ring': allowable_features['possible_is_in_ring_list'][is_in_ring_idx]
    }

    return feature_dict
# # uses same atom_feature as atom_to_feature_vector test
# atom_feature_dict = atom_feature_vector_to_dict(atom_feature)
# assert atom_feature_dict['atomic_num'] == 6
# assert atom_feature_dict['chirality'] == 'CHI_TETRAHEDRAL_CCW'
# assert atom_feature_dict['degree'] == 4
# assert atom_feature_dict['formal_charge'] == 0
# assert atom_feature_dict['num_h'] == 1
# assert atom_feature_dict['num_rad_e'] == 0
# assert atom_feature_dict['hybridization'] == 'SP3'
# assert atom_feature_dict['is_aromatic'] == False
# assert atom_feature_dict['is_in_ring'] == False

def bond_feature_vector_to_dict(bond_feature):
    [bond_type_idx, 
    bond_stereo_idx,
    is_conjugated_idx] = bond_feature

    feature_dict = {
        'bond_type': allowable_features['possible_bond_type_list'][bond_type_idx],
        'bond_stereo': allowable_features['possible_bond_stereo_list'][bond_stereo_idx],
        'is_conjugated': allowable_features['possible_is_conjugated_list'][is_conjugated_idx]
    }

    return feature_dict
# # uses same bond as bond_to_feature_vector test
# bond_feature_dict = bond_feature_vector_to_dict(bond_feature)
# assert bond_feature_dict['bond_type'] == 'DOUBLE'
# assert bond_feature_dict['bond_stereo'] == 'STEREOE'
# assert bond_feature_dict['is_conjugated'] == False

In [2]:
from ogb.utils.features import (allowable_features, atom_to_feature_vector,
 bond_to_feature_vector, atom_feature_vector_to_dict, bond_feature_vector_to_dict) 
from rdkit import Chem
import numpy as np


def ReorderCanonicalRankAtoms(mol):
    order = tuple(zip(*sorted([(j, i) for i, j in enumerate(Chem.CanonicalRankAtoms(mol))])))[1]
    mol_renum = Chem.RenumberAtoms(mol, order)
    return mol_renum, order

def smiles2graph(smiles_string, removeHs=True, reorder_atoms=False):
    """
    Converts SMILES string to graph Data object
    :input: SMILES string (str)
    :return: graph object
    """

    mol = Chem.MolFromSmiles(smiles_string)
    mol = mol if removeHs else Chem.AddHs(mol)
    if reorder_atoms:
        mol, _ = ReorderCanonicalRankAtoms(mol)

    # atoms
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom))
    x = np.array(atom_features_list, dtype = np.int64)

    # bonds
    num_bond_features = 3  # bond type, bond stereo, is_conjugated
    if len(mol.GetBonds()) > 0: # mol has bonds
        edges_list = []
        edge_features_list = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()

            edge_feature = bond_to_feature_vector(bond)

            # add edges in both directions
            edges_list.append((i, j))
            edge_features_list.append(edge_feature)
            edges_list.append((j, i))
            edge_features_list.append(edge_feature)

        # data.edge_index: Graph connectivity in COO format with shape [2, num_edges]
        edge_index = np.array(edges_list, dtype = np.int64).T

        # data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
        edge_attr = np.array(edge_features_list, dtype = np.int64)

    else:   # mol has no bonds
        edge_index = np.empty((2, 0), dtype = np.int64)
        edge_attr = np.empty((0, num_bond_features), dtype = np.int64)

    graph = dict()
    graph['edge_index'] = edge_index
    graph['edge_feat'] = edge_attr
    graph['node_feat'] = x
    graph['num_nodes'] = len(x)

    return graph 


if __name__ == '__main__':
    graph = smiles2graph('O1C=C[C@H]([C@H]1O2)c3c2cc(OC)c4c3OC(=O)C5=C4CCC(=O)5')
    print(graph)

{'edge_index': array([[ 0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  3,  6,  6,  7,  7,  8,
         8,  9,  9, 10, 10, 11,  9, 12, 12, 13, 13, 14, 14, 15, 15, 16,
        15, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22,  4,  0,  7,  5,
        13,  6, 18, 12, 21, 17],
       [ 1,  0,  2,  1,  3,  2,  4,  3,  5,  4,  6,  3,  7,  6,  8,  7,
         9,  8, 10,  9, 11, 10, 12,  9, 13, 12, 14, 13, 15, 14, 16, 15,
        17, 15, 18, 17, 19, 18, 20, 19, 21, 20, 22, 21,  0,  4,  5,  7,
         6, 13, 12, 18, 17, 21]]), 'edge_feat': array([[0, 0, 1],
       [0, 0, 1],
       [1, 0, 1],
       [1, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [3, 0, 1],
       [3, 0, 1],
       [3, 0, 1],
       [3, 0, 1],
       [3, 0, 1],
       [3, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [3, 0, 1],
       [3, 0, 1],
       [3, 0, 1],
       [3, 0, 1],
    

In [5]:
from rdkit import Chem

def graph2smiles(edge_index, edge_attr, x):
    """
    Converts graph data to a SMILES string.
    :param edge_index: Edge indices in COO format (shape [2, num_edges]).
    :param edge_attr: Edge features matrix (shape [num_edges, num_edge_features]).
    :param x: Node feature matrix (shape [num_nodes, num_node_features]).
    :return: SMILES string.
    """
    # Initialize an empty RDKit molecule
    mol = Chem.RWMol()

    # Add atoms to the molecule
    atom_indices = []
    for atom_features in x:
        atomic_num_idx = atom_features[0]  # First feature is atomic number
        atomic_num = allowable_features['possible_atomic_num_list'][atomic_num_idx]
        if atomic_num == 'misc':  # Handle invalid atomic number
            atomic_num = 0  # Assign Hydrogen as a placeholder
        atom_idx = mol.AddAtom(Chem.Atom(atomic_num))
        atom_indices.append(atom_idx)

    # Add bonds to the molecule
    num_edges = edge_index.shape[1]
    for edge_idx in range(num_edges):
        i, j = edge_index[:, edge_idx]  # Get atom indices for the bond
        bond_type_idx = edge_attr[edge_idx, 0]  # First edge feature is bond type
        bond_stereo_idx = edge_attr[edge_idx, 1]  # Second edge feature is bond stereo
        bond_type = allowable_features['possible_bond_type_list'][bond_type_idx]
        bond_stereo = allowable_features['possible_bond_stereo_list'][bond_stereo_idx]
        
        # Handle bond type
        if bond_type == 'SINGLE':
            bond = Chem.BondType.SINGLE
        elif bond_type == 'DOUBLE':
            bond = Chem.BondType.DOUBLE
        elif bond_type == 'TRIPLE':
            bond = Chem.BondType.TRIPLE
        elif bond_type == 'AROMATIC':
            bond = Chem.BondType.AROMATIC
        else:
            continue  # Skip invalid bonds

        # Add the bond
        if not mol.GetBondBetweenAtoms(int(i), int(j)):
            mol.AddBond(int(i), int(j), bond)
            bond_obj = mol.GetBondBetweenAtoms(int(i), int(j))
            
            # Handle bond stereo
            if bond_stereo == 'STEREOZ':
                bond_obj.SetStereo(Chem.BondStereo.STEREOZ)
            elif bond_stereo == 'STEREOE':
                bond_obj.SetStereo(Chem.BondStereo.STEREOE)
            elif bond_stereo == 'STEREOCIS':
                bond_obj.SetStereo(Chem.BondStereo.STEREOCIS)
            elif bond_stereo == 'STEREOTRANS':
                bond_obj.SetStereo(Chem.BondStereo.STEREOTRANS)

    # Generate the SMILES string
    mol.UpdatePropertyCache()
    Chem.SanitizeMol(mol)  # Ensure the molecule is valid
    smiles = Chem.MolToSmiles(mol, canonical=True)

    return smiles


In [8]:

graph = smiles2graph('O1C=C[C@H]([C@H]1O2)c3c2cc(OC)c4c3OC(=O)C5=C4CCC(=O)5')
smiles = graph2smiles(graph['edge_index'], graph['edge_feat'], graph['node_feat'])
print("Reconstructed SMILES:", smiles)


Reconstructed SMILES: COc1cc2c(c3oc(=O)c4c(c13)CCC4=O)C1C=COC1O2


In [11]:
from rdkit import Chem

original_smiles = 'c1ccccc1'

# Convert original SMILES to graph
graph = smiles2graph(original_smiles)

# Reconstruct SMILES from graph
reconstructed_smiles = graph2smiles(graph['edge_index'], graph['edge_feat'], graph['node_feat'])

# Check if the reconstructed SMILES is valid
reconstructed_mol = Chem.MolFromSmiles(reconstructed_smiles)
if reconstructed_mol:
    print("Reconstructed SMILES is valid.")
else:
    print("Reconstructed SMILES is invalid.")

# Normalize original and reconstructed SMILES using canonical SMILES
original_canonical = Chem.MolToSmiles(Chem.MolFromSmiles(original_smiles), canonical=True)
reconstructed_canonical = Chem.MolToSmiles(reconstructed_mol, canonical=True) if reconstructed_mol else None

print(f"Original SMILES (Canonical): {original_canonical}")
print(f"Reconstructed SMILES (Canonical): {reconstructed_canonical}")

# Compare molecules
if original_canonical == reconstructed_canonical:
    print("The original and reconstructed SMILES represent the same molecule.")
else:
    print("The original and reconstructed SMILES do NOT represent the same molecule.")


Reconstructed SMILES is valid.
Original SMILES (Canonical): c1ccccc1
Reconstructed SMILES (Canonical): c1ccccc1
The original and reconstructed SMILES represent the same molecule.
